In [14]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
#PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\catastro_S09_MIERCOLES.shp'
PATH_CAT = ''
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [15]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def marcar_lote_como_planificado(id_lote):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv SET estado = 'PLANIFICADO' WHERE id = :id_lote
            """)
            conn.execute(query, {"id_lote": id_lote})
            print(f"✔️ lote id={id_lote} planificado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_sin_planificar():
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where estado is null
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ No se pudo obtener parte_diario sin planificar. Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data



def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    gdf_pla_gral['mezcla'] = gdf_pla_gral['mezcla'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, mezcla, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', '{row['mezcla']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
        marcar_lote_como_planificado(row['id'])
    return None

def get_catastro():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM catastro_iag.catastro
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_catastro_adm():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM catastro_iag.catastro_adm
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

In [16]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [17]:
# obtiene el catastro agricola actual
gdf_cat = get_catastro()
# obtiene el catastro agricola actual
gdf_cat_adm = get_catastro_adm()
# obtiene los puntos planificados
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,procesado,mezcla,obs,fecha_registro
0,100,"MULTIPOINT (479748.509 8088589.344, 480268.204...",794,AGUILERA WENDE MARIANO,False,72,None,2026-02-12 22:52:06.766378


# CARGAR PLANIFICACION A PARTE DIARIO

In [18]:
# lista de todos los ids nuevos en planificacion
ids_planificacion = list(set(gdf_plan['id']))
ids_planificacion

[100]

In [19]:
# recorrer la lista de ids, seleccion uno de los registros de planificacion y los procesa para registrarlo en parte diario
for i in ids_planificacion:
    print(f"Procesando id: {i}")
    # filtra un registro de planificacion segun id
    plan = gdf_plan[gdf_plan['id'] == i]
    # intersecta registro con catastro_plan
    gdf_intersect_result = gpd.sjoin(gdf_cat_adm, plan, how='inner')
    if len(gdf_intersect_result) == 0:
        gdf_intersect_result = gpd.sjoin(gdf_cat, plan, how='inner')
    if len(gdf_intersect_result) == 0:
        continue
    # selecciona la columnas de interes
    gdf_result = gdf_intersect_result[['geom', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona', 'mezcla']].copy()
    # recalcula en area
    gdf_result['area'] = gdf_result.geometry.area / 10000
    # renombra las columnas
    gdf_result.rename(columns={
        'geometry': 'geom',
        'codigo_canero': 'unidad_03',
        'nombre_canero': 'unidad_04',
        'id_right': 'os',
        'zona': 'inst'
    }, inplace=True)
    # asigna la columna de geometria
    gdf_result = gdf_result.set_geometry("geom")
    # cambia tipos de datos de columnas
    gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
    gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
    gdf_result['os'] = gdf_result['os'].astype(int)
    gdf_result['soca'] = gdf_result['soca'].astype(int)
    gdf_result['inst'] = gdf_result['inst'].astype(int)
    gdf_result['mezcla'] = gdf_result['mezcla'].astype(int)
    
    # GARANTIZAR LA INST DEL CAÑERO
    # cargar grupos de cosecha
    df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
    df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
    df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
    df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)
    # Crear un diccionario de mapeo: {codigo_cañero: institucion}
    mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
    # Reemplazar los valores de 'inst' en gdf_result usando el diccionario
    # a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
    gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)
    
    # agregar nuevos registros a la base de datos
    gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")
    marcar_como_procesado(i)

Procesando id: 100
✔️ id 100 marcado como procesado.


# CARGAR LOTES A AMIGOCLOUD

In [20]:
# obtener parte diario sin planificar
parte_diario_sin_planificar = obtener_parte_diario_sin_planificar()
# lista de os del parte diario son planificar
lista_os_sin_planificar = list(set(parte_diario_sin_planificar['os']))

In [21]:
# CARGAR A AMIGOCLOD
# recorriendo la lista de os
for i in lista_os_sin_planificar:
    plan_os = parte_diario_sin_planificar[parte_diario_sin_planificar['os'] == i]
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
✔️ lote id=897 planificado.
Lote registrado en AmigoCloud
✔️ lote id=898 planificado.
Lote registrado en AmigoCloud
✔️ lote id=899 planificado.
Lote registrado en AmigoCloud
✔️ lote id=900 planificado.
Lote registrado en AmigoCloud
✔️ lote id=901 planificado.
Lote registrado en AmigoCloud
✔️ lote id=902 planificado.
Lote registrado en AmigoCloud
✔️ lote id=903 planificado.
Lote registrado en AmigoCloud
✔️ lote id=904 planificado.
Lote registrado en AmigoCloud
✔️ lote id=905 planificado.
Lote registrado en AmigoCloud
✔️ lote id=906 planificado.
Lote registrado en AmigoCloud
✔️ lote id=907 planificado.
Lote registrado en AmigoCloud
✔️ lote id=908 planificado.
Lote registrado en AmigoCloud
✔️ lote id=909 planificado.
Lote registrado en AmigoCloud
✔️ lote id=910 planificado.
Lote registrado en AmigoCloud
✔️ lote id=911 planificado.
Lote registrado en AmigoCloud
✔️ lote id=912 planificado.
Lote registrado en AmigoCloud
✔️ lote id=913 planificado.
Lote registrad